# Get 99 Ranch Market locations

#### Load Python tools and Jupyter config

In [1]:
import us 
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
place = "99ranch"
place_formal = "99 Ranch Market"
color = "#2a6f50"

## Scrape

#### Headers, params and request for locations

In [4]:
headers = {
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "time-zone": "America/Los_Angeles",
}

In [5]:
json_data = {
    "zipCode": "90066",
    "pageSize": 100,
    "pageNum": 1,
    "type": 1,
    "source": "WEB",
    "within": None,
}

In [6]:
response = requests.post(
    "https://api.awsprod.99ranch.com/store/web/nearby/stores",
    headers=headers,
    json=json_data,
)

#### All the locations are returned without pagination. Put into a dataframe. 

In [7]:
df = pd.DataFrame(response.json()["data"]["records"])[
    [
        "storeNumber",
        "name",
        "state",
        "city",
        "street",
        "zipCode",
        "longitude",
        "latitude",
        "timeZone",
    ]
].rename(
    columns={
        "storeNumber": "store_number",
        "zipCode": "zip",
        "name": "store_name",
        "telephone": "phone",
    }
)

#### The result and count

In [8]:
df.head()

,store_number,store_name,state,city,street,zip,longitude,latitude,timeZone
0,1255,Westwood,CA,Los Angeles,1360 Westwood Blvd,90024,-118.441905700000,34.055985800000,America/Los_Angeles
1,1006,Gardena,CA,Gardena,1340 Artesia Blvd,90248,-118.297151500000,33.872055000000,America/Los_Angeles
2,1808,Torrance,CA,Torrance,19100 Harborgate Way,90501,-118.303001700000,33.857125700000,America/Los_Angeles
3,1013,Van Nuys,CA,Van Nuys,6450 Sepulveda Blvd,91411,-118.465335900000,34.188030900000,America/Los_Angeles
4,1012,Monterey Park,CA,Monterey Park,771 W Garvey Ave,91754,-118.133740900000,34.063652400000,America/Los_Angeles


In [9]:
len(df)

62

#### Create a mapping of state abbreviations to full state names using the us library

In [10]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [11]:
df["state_name"] = df["state"].map(state_mapping)

#### Make sure our brand name gets in the dataframe

In [12]:
df["brand"] = place_formal

---

## Geography

#### Make it a geodataframe

In [13]:
df_geo = df.copy()

In [14]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

---

## Maps

#### US states background

In [15]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [16]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=15, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [17]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [18]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [19]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [20]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)